In [1]:
%config InlineBackend.figure_format = 'retina'
%pylab inline
import sys
sys.path.insert(1, '../codes/')
from geeCodes import *

import ee
#earthengine authenticate
ee.Authenticate()
ee.Initialize()

Populating the interactive namespace from numpy and matplotlib


Enter verification code: 4/1AWtgzh6osPKuCqGoDbdDhPsz_Xrw_HAm0SPZygZh1SC9I20xAAgI7pNyLKg

Successfully saved authorization token.


In [82]:
########################## Read city file ###########################################
dfc = gpd.read_file('../data_revision/cities/all/gdfCities.shp')
dfc = dfc.to_crs("EPSG:4326")

In [83]:
cities = ['Toronto', 'Jeddah', 'Bogota']

In [84]:
#### Toronto
dfc[dfc['UC_NM_MN']==cities[0]]

,AREA,UC_NM_MN,UC_NM_LST,CTR_MN_ISO,E_KG_NM_LS,P15,idx,climate,PopDens,geometry
24,2019.0,Toronto,Toronto; Mississauga; Brampton; Hamilton; Mark...,CAN,"Snow, fully humid, and Warm summer",6.036146e+06,24,Continental,2989.671017,"POLYGON ((-79.45216 43.96878, -79.43991 43.968..."


In [85]:
#### Jeddah
dfc[dfc['UC_NM_MN']==cities[1]]

,AREA,UC_NM_MN,UC_NM_LST,CTR_MN_ISO,E_KG_NM_LS,P15,idx,climate,PopDens,geometry
137,537.0,Jeddah,Jeddah,SAU,"Desert (arid), and Hot arid",3.517162e+06,143,Arid,6549.649384,"POLYGON ((39.07593 21.72154, 39.11770 21.72154..."


In [86]:
#### Bogota
dfc[dfc['UC_NM_MN']==cities[2]]

,AREA,UC_NM_MN,UC_NM_LST,CTR_MN_ISO,E_KG_NM_LS,P15,idx,climate,PopDens,geometry
136,538.0,Bogota,Bogota; Soacha,COL,"Mild temperate, fully humid, and Warm summer",8.608815e+06,142,Arid,16001.51564,"POLYGON ((-74.05724 4.82244, -74.02724 4.82244..."


In [87]:
def maskClouds(image):
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(1 << 5).eq(0)
    return image.updateMask(mask)
folder = 'terrain30'

In [95]:
geo = dfc[dfc['UC_NM_MN']==cities[0]].reset_index(drop=True)
S = returnCityBoundary(geo)
####### GEE Geometry ###############################################
geometry = returnGeometry(S[0])


landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .filterDate('2010-01-01', '2020-12-31') \
           .filterBounds(geometry) \
           .filter(ee.Filter.lt('CLOUD_COVER', 10))

landsat_masked = landsat.map(maskClouds)
landsat_mean = ee.Image(landsat_masked.select(['B4', 'B3', 'B2']).mean()).clip(geometry);

dscr = 'Toronto3'
print(dscr)
task = ee.batch.Export.image.toDrive(image=landsat_mean,  # an ee.Image object.
                                     region=geometry,  # an ee.Geometry object.
                                     description=dscr,
                                     folder=folder,
                                     fileNamePrefix=dscr,
                                     scale = 100,
                                     crs = 'epsg:4326')
task.start()
while task.status()['state'] != 'COMPLETED':
    print(task.status()['state'])

Toronto3
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
READY
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
